In [1]:
import json
from shapely.wkt import loads
from multiprocessing import Pool
import sys
from os import path, makedirs, listdir
import timeit
import cv2
import random
import numpy as np
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

np.random.seed(1)
random.seed(1)
sys.setrecursionlimit(10000)
masks_dir = 'masks'
###To be changed####
train_dirs = ['C:/Users/liuyi/segment/ubdd/xbd/train',
              'C:/Users/liuyi/segment/ubdd/xbd/test',
              'C:/Users/liuyi/segment/ubdd/xbd/tier3',
              'C:/Users/liuyi/segment/ubdd/xbd/hold']# 加了验证集
#train_dirs = ['/local_storage/datasets/sgerard/xview2/train',
#               '/local_storage/datasets/sgerard/xview2/test',
#               '/local_storage/datasets/sgerard/xview2/tier3']

In [3]:
def mask_for_polygon(poly, im_size=(1024, 1024)):
    img_mask = np.zeros(im_size, np.uint8)
    def int_coords(x): return np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords)]
    interiors = [int_coords(pi.coords) for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask
#对于 4 个有效类别 + 一个背景类别，类别数量应为 5。
damage_dict = {
    "no-damage": 1,
    "minor-damage": 2,
    "major-damage": 3,
    "destroyed": 4,
    "un-classified": 0  # ? 有的会用255
}
#如果您要进行CrossEntropyLoss多类别分割，则掩码中的值应表示从 0 到 的类别索引num_classes - 1。例如：
# 无损坏”：0
# 轻微损坏”：1
# 重大损害”：2
# 已销毁”：3
# 直接使用 50、100、150、255 等值可能会导致问题，因为CrossEntropyLoss这些值是类索引，而不是任意的大数字。
# damage_dict = {
#     "no-damage": 50,
#     "minor-damage": 100,
#     "major-damage": 150,
#     "destroyed": 255,
#     "un-classified": 50  # ?
# }
def process_image(json_file):
    print(f"Processing: {json_file}")
    js1 = json.load(open(json_file))
    js2 = json.load(open(json_file.replace('_pre_disaster', '_post_disaster')))

    msk = np.zeros((1024, 1024), dtype='uint8')
    msk_damage = np.zeros((1024, 1024), dtype='uint8')

    for feat in js1['features']['xy']:
        poly = loads(feat['wkt'])
        _msk = mask_for_polygon(poly)
        msk[_msk > 0] = 1
        # msk[_msk > 0] = 255

    for feat in js2['features']['xy']:
        poly = loads(feat['wkt'])
        subtype = feat['properties']['subtype']
        _msk = mask_for_polygon(poly)
        msk_damage[_msk > 0] = damage_dict[subtype]

    pre_disaster_path = json_file.replace('labels', 'masks').replace('_pre_disaster.json', '_pre_disaster.png')
    post_disaster_path = json_file.replace('labels', 'masks').replace('_pre_disaster.json', '_post_disaster.png')

    # 确保目标文件夹存在
    makedirs(path.dirname(pre_disaster_path), exist_ok=True)
    makedirs(path.dirname(post_disaster_path), exist_ok=True)

    # 保存文件
    cv2.imwrite(pre_disaster_path, msk, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    cv2.imwrite(post_disaster_path, msk_damage, [cv2.IMWRITE_PNG_COMPRESSION, 9])

if __name__ == '__main__':
    t0 = timeit.default_timer()

    all_files = []
    for d in train_dirs:
        makedirs(path.join(d, masks_dir), exist_ok=True)
        for f in sorted(listdir(path.join(d, 'images'))):
            if '_pre_disaster.png' in f:
                all_files.append(path.join(d, 'labels', f.replace('_pre_disaster.png', '_pre_disaster.json')))
                
    print(f"Found {len(all_files)} files to process")

    for file in all_files:
        process_image(file)

    elapsed = timeit.default_timer() - t0
    print('Time: {:.3f} min'.format(elapsed / 60))

Found 933 files to process
Processing: C:/Users/liuyi/segment/ubdd/xbd/hold\labels\guatemala-volcano_00000004_pre_disaster.json
Processing: C:/Users/liuyi/segment/ubdd/xbd/hold\labels\guatemala-volcano_00000012_pre_disaster.json
Processing: C:/Users/liuyi/segment/ubdd/xbd/hold\labels\guatemala-volcano_00000014_pre_disaster.json
Processing: C:/Users/liuyi/segment/ubdd/xbd/hold\labels\guatemala-volcano_00000020_pre_disaster.json
Processing: C:/Users/liuyi/segment/ubdd/xbd/hold\labels\guatemala-volcano_00000022_pre_disaster.json
Processing: C:/Users/liuyi/segment/ubdd/xbd/hold\labels\hurricane-florence_00000006_pre_disaster.json
Processing: C:/Users/liuyi/segment/ubdd/xbd/hold\labels\hurricane-florence_00000009_pre_disaster.json
Processing: C:/Users/liuyi/segment/ubdd/xbd/hold\labels\hurricane-florence_00000010_pre_disaster.json
Processing: C:/Users/liuyi/segment/ubdd/xbd/hold\labels\hurricane-florence_00000011_pre_disaster.json
Processing: C:/Users/liuyi/segment/ubdd/xbd/hold\labels\hurr

In [5]:



def mask_for_polygon(poly, im_size=(1024, 1024)):
    img_mask = np.zeros(im_size, np.uint8)
    def int_coords(x): return np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords)]
    interiors = [int_coords(pi.coords) for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask


damage_dict = {
    "no-damage": 1,
    "minor-damage": 2,
    "major-damage": 3,
    "destroyed": 4,
    "un-classified": 1  # ?
}


def process_image(json_file):
    print(f"Processing: {json_file}")
    js1 = json.load(open(json_file))
    js2 = json.load(open(json_file.replace('_pre_disaster', '_post_disaster')))

    msk = np.zeros((1024, 1024), dtype='uint8')
    msk_damage = np.zeros((1024, 1024), dtype='uint8')

    for feat in js1['features']['xy']:
        poly = loads(feat['wkt'])
        _msk = mask_for_polygon(poly)
        msk[_msk > 0] = 255

    for feat in js2['features']['xy']:
        poly = loads(feat['wkt'])
        subtype = feat['properties']['subtype']
        _msk = mask_for_polygon(poly)
        msk_damage[_msk > 0] = damage_dict[subtype]

    saved_pre = cv2.imwrite(json_file.replace('/labels/', '/masks/').replace('_pre_disaster.json',
                '_pre_disaster.png'), msk, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    saved_post = cv2.imwrite(json_file.replace('/labels/', '/masks/').replace('_pre_disaster.json',
                '_post_disaster.png'), msk_damage, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    print(f"Pre-disaster mask saved: {saved_pre}")
    print(f"Post-disaster mask saved: {saved_post}")

if __name__ == '__main__':
    t0 = timeit.default_timer()

    all_files = []
    for d in train_dirs:
        makedirs(path.join(d, masks_dir), exist_ok=True)
        for f in sorted(listdir(path.join(d, 'images'))):
            if '_pre_disaster.png' in f:
                all_files.append(path.join(d, 'labels', f.replace(
                    '_pre_disaster.png', '_pre_disaster.json')))
                
    print(f"Found {len(all_files)} files to process")
    
    for file in all_files:
        process_image(file)
    # Pool()在windows下或者jupyter下会无线循环
    # with Pool() as pool:
    #     _ = pool.map(process_image, all_files)

    elapsed = timeit.default_timer() - t0
    print('Time: {:.3f} min'.format(elapsed / 60))

Found 10101 files to process
Processing: C:/Users/liuyi/segment/ubdd/xbd/train\labels\guatemala-volcano_00000000_pre_disaster.json
Pre-disaster mask saved: True
Post-disaster mask saved: True
Processing: C:/Users/liuyi/segment/ubdd/xbd/train\labels\guatemala-volcano_00000001_pre_disaster.json
Pre-disaster mask saved: True
Post-disaster mask saved: True
Processing: C:/Users/liuyi/segment/ubdd/xbd/train\labels\guatemala-volcano_00000002_pre_disaster.json
Pre-disaster mask saved: True
Post-disaster mask saved: True
Processing: C:/Users/liuyi/segment/ubdd/xbd/train\labels\guatemala-volcano_00000006_pre_disaster.json
Pre-disaster mask saved: True
Post-disaster mask saved: True
Processing: C:/Users/liuyi/segment/ubdd/xbd/train\labels\guatemala-volcano_00000007_pre_disaster.json
Pre-disaster mask saved: True
Post-disaster mask saved: True
Processing: C:/Users/liuyi/segment/ubdd/xbd/train\labels\guatemala-volcano_00000008_pre_disaster.json
Pre-disaster mask saved: True
Post-disaster mask saved

KeyboardInterrupt: 

In [ ]:
{"features": {"lng_lat": [{"properties": {"feature_type": "building", "uid": "486b0813-ecd2-4b84-856c-9c0e42156953"}, "wkt": "POLYGON ((-90.8154467949004 14.39086318334907, -90.81590706308027 14.39071123556949, -90.81587964154231 14.39059650626618, -90.81595559688809 14.3905736709202, -90.8159334443047 14.39048145754321, -90.81586635209149 14.39049581582651, -90.81584174451078 14.39043032648, -90.81537467349251 14.39060467857228, -90.8154467949004 14.39086318334907))"}, {"properties": {"feature_type": "building", "uid": "139cf2c8-ad52-4739-82b5-bb646b215e76"}, "wkt": "POLYGON ((-90.81420592026568 14.38829423654955, -90.81421178335277 14.38832937106096, -90.81408837763034 14.38834314934303, -90.81404547542751 14.38816089961473, -90.81410016817848 14.38815172795478, -90.8141122317117 14.38818732301418, -90.81427959936715 14.38816185446882, -90.81428289440765 14.38820583612159, -90.81412306221677 14.38822210022975, -90.81413260646352 14.38830404251541, -90.8141417336975 14.38830130434685, -90.81414222186321 14.388293647644, -90.81416210337215 14.38829100890138, -90.81416453412773 14.38829627325912, -90.81420592026568 14.38829423654955))"}, {"properties": {"feature_type": "building", "uid": "d43deb4a-529c-4df4-b666-26dd5b17e040"}, "wkt": "POLYGON ((-90.81324060670399 14.38779112489009, -90.81333271039148 14.38777463819341, -90.81334010069584 14.38778035164649, -90.81339474233999 14.38777642155136, -90.81339470309663 14.38778045340581, -90.81343276442996 14.38777999895865, -90.81345343078702 14.38790922104149, -90.81323921345755 14.38793425570734, -90.81324060670399 14.38779112489009))"}, {"properties": {"feature_type": "building", "uid": "563b145d-732d-4eb4-8c77-380519842324"}, "wkt": "POLYGON ((-90.81324307540712 14.38753750651894, -90.81325659681856 14.38760208792226, -90.81333765145776 14.38758181977528, -90.81328602968085 14.38741648412458, -90.81324416128489 14.38742594898786, -90.81324307540712 14.38753750651894))"}, {"properties": {"feature_type": "building", "uid": "56f51b26-d511-461d-bb31-747901a4ea75"}, "wkt": "POLYGON ((-90.81335464228361 14.38761690782337, -90.81351339533825 14.38755883129817, -90.81344244089756 14.38734722529422, -90.81349861418137 14.38733272479192, -90.81348240401722 14.3872711295457, -90.81342760484934 14.38728837365763, -90.81336870460227 14.38715128375731, -90.81324635403672 14.38720067395213, -90.8132456098203 14.3872771323734, -90.81335464228361 14.38761690782337))"}, {"properties": {"feature_type": "building", "uid": "7fd91a32-cf6c-43f2-b059-8c77c92a5693"}, "wkt": "POLYGON ((-90.81360818448977 14.38704360849436, -90.8136164330836 14.38706401243825, -90.81359363322092 14.38707409410125, -90.81360334282276 14.38710711985712, -90.81352441525669 14.38713417973061, -90.81348782327485 14.3870424954982, -90.81360818448977 14.38704360849436))"}, {"properties": {"feature_type": "building", "uid": "aa0e6190-a04e-427b-bcb1-9a518e4ddba7"}, "wkt": "POLYGON ((-90.81555089005624 14.38950312803611, -90.81569758649049 14.38945459697502, -90.81566908337692 14.38937831608995, -90.81554677864288 14.38942469697396, -90.81552482384659 14.38942686965674, -90.81555089005624 14.38950312803611))"}, {"properties": {"feature_type": "building", "uid": "c9128d2c-3ef9-46a4-b6e1-f9c66ad53a29"}, "wkt": "POLYGON ((-90.81577300617678 14.39121026372967, -90.81579500699497 14.39125567680618, -90.81583259210578 14.39123706514994, -90.81580761340278 14.39119016618024, -90.81577300617678 14.39121026372967))"}, {"properties": {"feature_type": "building", "uid": "a52e5ed2-8e9a-422e-98cd-5e0e6e2cb72c"}, "wkt": "POLYGON ((-90.81602025678738 14.3916040869195, -90.8160100042802 14.39158399663942, -90.81595446961606 14.39160347908017, -90.81602025678738 14.3916040869195))"}, {"properties": {"feature_type": "building", "uid": "ec1c17b7-b24d-4723-9f1c-0e0c7f83eeaa"}, "wkt": "POLYGON ((-90.81367722212008 14.38908618490269, -90.81374413297719 14.38907838321822, -90.81373420320192 14.38899305277918, -90.81365847145507 14.3890040228894, -90.81367722212008 14.38908618490269))"}], "xy": [{"properties": {"feature_type": "building", "uid": "486b0813-ecd2-4b84-856c-9c0e42156953"}, "wkt": "POLYGON ((532.1863731987335 165.9824500704975, 431.2786444340596 201.2277333232305, 437.5552017256422 227.0581806385896, 420.8981842979807 232.3691137314672, 425.967711341182 253.130034003625, 440.6934803714335 249.7503493081575, 446.2458195139873 264.4761183384089, 548.6019845767187 224.1613080424746, 532.1863731987335 165.9824500704975))"}, {"properties": {"feature_type": "building", "uid": "139cf2c8-ad52-4739-82b5-bb646b215e76"}, "wkt": "POLYGON ((810.6063105745833 743.0540226914463, 809.2415595562098 735.1384667848803, 836.3546131212291 731.7720809395591, 846.180820453518 772.8055948919877, 834.1710114918316 774.9891965213851, 831.4415094550847 766.9826572135942, 794.6842153602263 773.0785450956623, 793.8653647492023 763.1613543621486, 828.9849576220124 759.1580847082531, 826.7103725913901 740.688454259599, 824.7087377644423 741.3253380681733, 824.6177543632174 743.0540226914463, 820.2505511044224 743.6909065000207, 819.7046506970729 742.508122284097, 810.6063105745833 743.0540226914463))"}, {"properties": {"feature_type": "building", "uid": "d43deb4a-529c-4df4-b666-26dd5b17e040"}, "wkt": "POLYGON ((1024 854.5625520879265, 1003.777438895615 858.4748383405971, 1002.139737673567 857.2010707234485, 990.1299287118808 858.2018881369223, 990.1299287118808 857.2920541246734, 981.7594557991903 857.4740209271232, 976.9373355342708 828.3593325351562, 1024 822.2634446530881, 1024 854.5625520879265))"}, {"properties": {"feature_type": "building", "uid": "563b145d-732d-4eb4-8c77-380519842324"}, "wkt": "POLYGON ((1024 911.7944398581552, 1020.887750158995 897.2504368870575, 1003.103465353255 901.992912835255, 1014.811452850367 939.1917085539287, 1024 936.9686729532111, 1024 911.7944398581552))"}, {"properties": {"feature_type": "building", "uid": "56f51b26-d511-461d-bb31-747901a4ea75"}, "wkt": "POLYGON ((999.2912769870155 894.1110629984253, 964.4984425398568 907.5467864893407, 980.5576052708358 955.1457395442582, 968.2335216597253 958.5348625373135, 971.9307467430584 972.3994565998128, 983.9467282638911 968.3941294262019, 997.1951181458348 999.2043384539779, 1024 987.8045611137007, 1024 970.5508440581461, 999.2912769870155 894.1110629984253))"}, {"properties": {"feature_type": "building", "uid": "7fd91a32-cf6c-43f2-b059-8c77c92a5693"}, "wkt": "POLYGON ((944.7527593214082 1024, 942.8948323217783 1019.413242719663, 947.888011133284 1017.090833970126, 945.6817228212233 1009.659125971606, 962.9836680052778 1003.388622347855, 971.2282190661359 1024, 944.7527593214082 1024))"}, {"properties": {"feature_type": "building", "uid": "aa0e6190-a04e-427b-bcb1-9a518e4ddba7"}, "wkt": "POLYGON ((512.2005986924418 473.0835632764903, 480.0393628206124 484.3399958316305, 486.4716099949783 501.4926549632729, 513.2726398881695 490.7722430059964, 518.0968252689438 490.2362224081326, 512.2005986924418 473.0835632764903))"}, {"properties": {"feature_type": "building", "uid": "c9128d2c-3ef9-46a4-b6e1-f9c66ad53a29"}, "wkt": "POLYGON ((459.6961333932207 88.34711072680092, 454.7600811552291 78.14593610161822, 446.533327425243 82.42384804121096, 452.1275199616335 92.95409281559311, 459.6961333932207 88.34711072680092))"}, {"properties": {"feature_type": "building", "uid": "a52e5ed2-8e9a-422e-98cd-5e0e6e2cb72c"}, "wkt": "POLYGON ((404.4729182033129 0, 406.7708453697031 4.511813209056069, 418.9435881332131 0, 404.4729182033129 0))"}, {"properties": {"feature_type": "building", "uid": "ec1c17b7-b24d-4723-9f1c-0e0c7f83eeaa"}, "wkt": "POLYGON ((925.1958625119616 563.2548991674288, 910.4958625119616 565.1548991674288, 912.8625291786283 584.3882325007621, 929.4958625119616 581.7548991674288, 925.1958625119616 563.2548991674288))"}]}, "metadata": {"sensor": "WORLDVIEW03_VNIR", "provider_asset_type": "WORLDVIEW03_VNIR", "gsd": 1.3459189, "capture_date": "2018-02-05T17:10:18.000Z", "off_nadir_angle": 17.933279, "pan_resolution": 0.3362646, "sun_azimuth": 150.80763000000002, "sun_elevation": 55.506645, "target_azimuth": 118.24727, "disaster": "guatemala-volcano", "disaster_type": "volcano", "catalog_id": "10400100377A9B00", "original_width": 1024, "original_height": 1024, "width": 1024, "height": 1024, "id": "MjUxMzEyMA.FwIEh3iVRb5-R_EXW9amgViNSoA", "img_name": "guatemala-volcano_00000000_pre_disaster.png"}}

In [ ]:

{"features": {"lng_lat": [{"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "486b0813-ecd2-4b84-856c-9c0e42156953"}, "wkt": "POLYGON ((-90.81544679490855 14.39086318334812, -90.81537467350067 14.39060467857134, -90.81584174451893 14.39043032647906, -90.81586635209965 14.39049581582557, -90.81593344431286 14.39048145754227, -90.81595559689623 14.39057367091926, -90.81587964155047 14.39059650626524, -90.81590706308843 14.39071123556855, -90.81544679490855 14.39086318334812))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "139cf2c8-ad52-4739-82b5-bb646b215e76"}, "wkt": "POLYGON ((-90.81420592027385 14.38829423654861, -90.81421178336095 14.38832937106001, -90.81408837763853 14.38834314934208, -90.81404547543569 14.38816089961379, -90.81410016818667 14.38815172795384, -90.8141122317199 14.38818732301324, -90.81427959937533 14.38816185446788, -90.81428289441583 14.38820583612065, -90.81412306222495 14.38822210022881, -90.8141326064717 14.38830404251447, -90.81414173370567 14.38830130434591, -90.81414222187138 14.38829364764306, -90.81416210338033 14.38829100890043, -90.81416453413591 14.38829627325817, -90.81420592027385 14.38829423654861))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "d43deb4a-529c-4df4-b666-26dd5b17e040"}, "wkt": "POLYGON ((-90.81324289816379 14.3877907147159, -90.81333271039968 14.38777463819247, -90.81334010070404 14.38778035164554, -90.81339474234819 14.38777642155041, -90.81339470310483 14.38778045340487, -90.81343276443816 14.38777999895771, -90.81345343079522 14.38790922104054, -90.81324150337852 14.38793398809405, -90.81324289816379 14.3877907147159))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "563b145d-732d-4eb4-8c77-380519842324"}, "wkt": "POLYGON ((-90.81324645425258 14.38742543063536, -90.81328602968905 14.38741648412363, -90.81333765146596 14.38758181977434, -90.81325659682676 14.38760208792131, -90.8132452615307 14.38754794791308, -90.81324645425258 14.38742543063536))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "56f51b26-d511-461d-bb31-747901a4ea75"}, "wkt": "POLYGON ((-90.81324865131654 14.38719974659386, -90.81336870461047 14.38715128375636, -90.81342760485754 14.38728837365668, -90.81348240402542 14.38727112954476, -90.81349861418958 14.38733272479098, -90.81344244090576 14.38734722529327, -90.81351339534643 14.38755883129722, -90.81335464229181 14.38761690782242, -90.81324783058389 14.38728405286323, -90.81324865131654 14.38719974659386))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "7fd91a32-cf6c-43f2-b059-8c77c92a5693"}, "wkt": "POLYGON ((-90.81361001823072 14.38704814446417, -90.81361643309178 14.38706401243731, -90.81359363322912 14.38707409410031, -90.81360334283094 14.38710711985617, -90.81352441526488 14.38713417972967, -90.81348963395479 14.38704703228479, -90.81361001823072 14.38704814446417))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "aa0e6190-a04e-427b-bcb1-9a518e4ddba7"}, "wkt": "POLYGON ((-90.8155508900644 14.38950312803517, -90.81552482385474 14.38942686965579, -90.81554677865103 14.38942469697301, -90.81566908338507 14.389378316089, -90.81569758649864 14.38945459697408, -90.8155508900644 14.38950312803517))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "c9128d2c-3ef9-46a4-b6e1-f9c66ad53a29"}, "wkt": "POLYGON ((-90.81577300618494 14.39121026372873, -90.81580761341094 14.39119016617929, -90.81583259211392 14.391237065149, -90.81579500700313 14.39125567680525, -90.81577300618494 14.39121026372873))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "a52e5ed2-8e9a-422e-98cd-5e0e6e2cb72c"}, "wkt": "POLYGON ((-90.81595454442756 14.39160345283703, -90.81601000428834 14.39158399663848, -90.81602024305114 14.3916040599858, -90.81595454442756 14.39160345283703))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "ec1c17b7-b24d-4723-9f1c-0e0c7f83eeaa"}, "wkt": "POLYGON ((-90.81367722212828 14.38908618490175, -90.81365847146327 14.38900402288845, -90.81373420321012 14.38899305277823, -90.81374413298538 14.38907838321727, -90.81367722212828 14.38908618490175))"}], "xy": [{"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "486b0813-ecd2-4b84-856c-9c0e42156953"}, "wkt": "POLYGON ((532.1863714054954 165.9824503000504, 548.6019827848852 224.1613082732768, 446.2458177229872 264.4761185678668, 440.6934785817455 249.7503495372735, 425.9677095499687 253.1300342329752, 420.8981825090282 232.3691139606, 437.5551999358392 227.0581808684396, 431.2786426428399 201.2277335525121, 532.1863714054954 165.9824503000504))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "139cf2c8-ad52-4739-82b5-bb646b215e76"}, "wkt": "POLYGON ((810.6063087761681 743.0540229226118, 809.2415577592562 735.1384670145518, 836.3546113250679 731.7720811707778, 846.1808186532426 772.8055951221552, 834.1710096970594 774.9891967511136, 831.4415076554028 766.982657443192, 794.6842135649372 773.0785453250629, 793.8653629500429 763.16135459204, 828.9849558247239 759.158084939266, 826.7103707925814 740.6884544906072, 824.7087359669069 741.3253382986578, 824.617752566808 743.0540229221372, 820.2505493091294 743.6909067292383, 819.7046488990417 742.5081225153722, 810.6063087761681 743.0540229226118))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "d43deb4a-529c-4df4-b666-26dd5b17e040"}, "wkt": "POLYGON ((1023.496880254201 854.6598863601454, 1003.77743709242 858.4748385709993, 1002.139735873313 857.2010709534738, 990.1299269123824 858.2018883676168, 990.1299269126198 857.2920543547596, 981.7594539971835 857.4740211578057, 976.9373337333138 828.3593327648889, 1023.496913822461 822.3286081604058, 1023.496880254201 854.6598863601454))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "563b145d-732d-4eb4-8c77-380519842324"}, "wkt": "POLYGON ((1023.496779355264 937.0904197282952, 1014.811451048506 939.1917087852462, 1003.103463552428 901.9929130662301, 1020.887748356705 897.250437118042, 1023.496815649079 909.4429941108469, 1023.496779355264 937.0904197282952))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "56f51b26-d511-461d-bb31-747901a4ea75"}, "wkt": "POLYGON ((1023.49670602115 988.0186053329112, 997.1951163476003 999.204338684292, 983.9467264620505 968.3941296577275, 971.9307449447311 972.3994568308011, 968.2335198583162 958.5348627683724, 980.5576034682273 955.1457397747856, 964.4984407419494 907.5467867202947, 999.2912751836308 894.1110632298545, 1023.496734396069 968.9939313664818, 1023.49670602115 988.0186053329112))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "7fd91a32-cf6c-43f2-b059-8c77c92a5693"}, "wkt": "POLYGON ((944.3397244786014 1022.980324969038, 942.8948305239735 1019.413242950562, 947.8880093326121 1017.090834200889, 945.6817210196876 1009.659126203455, 962.9836662078554 1003.388622578721, 970.8202541150114 1022.980092689001, 944.3397244786014 1022.980324969038))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "aa0e6190-a04e-427b-bcb1-9a518e4ddba7"}, "wkt": "POLYGON ((512.2005969012966 473.0835635061429, 518.0968234767244 490.2362226368645, 513.2726380997851 490.7722432348034, 486.4716082016596 501.4926551931065, 480.0393610282932 484.3399960609608, 512.2005969012966 473.0835635061429))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "c9128d2c-3ef9-46a4-b6e1-f9c66ad53a29"}, "wkt": "POLYGON ((459.6961316036035 88.34711095609332, 452.1275181710914 92.95409304580041, 446.5333256343371 82.42384827095465, 454.7600793633781 78.14593633058018, 459.6961316036035 88.34711095609332))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "a52e5ed2-8e9a-422e-98cd-5e0e6e2cb72c"}, "wkt": "POLYGON ((418.9271900639632 0.006077493789786598, 406.770843578674 4.511813438970321, 404.4759969843418 0.006048706821852865, 418.9271900639632 0.006077493789786598))"}, {"properties": {"feature_type": "building", "subtype": "no-damage", "uid": "ec1c17b7-b24d-4723-9f1c-0e0c7f83eeaa"}, "wkt": "POLYGON ((925.1958607115536 563.2548993988472, 929.4958607114152 581.7548993980023, 912.8625273773621 584.3882327314795, 910.4958607157192 565.1548993980138, 925.1958607115536 563.2548993988472))"}]}, "metadata": {"sensor": "WORLDVIEW03_VNIR", "provider_asset_type": "WORLDVIEW03_VNIR", "gsd": 1.4085245, "capture_date": "2018-06-22T16:55:40.000Z", "off_nadir_angle": 21.97336, "pan_resolution": 0.35173947, "sun_azimuth": 58.571484, "sun_elevation": 71.182274, "target_azimuth": 314.29373, "disaster": "guatemala-volcano", "disaster_type": "volcano", "catalog_id": "104001003E575F00", "original_width": 1024, "original_height": 1024, "width": 1024, "height": 1024, "id": "MjU0NzE2NA.73QhG8U-LJLGkyNS0FB24X8vQS8", "img_name": "guatemala-volcano_00000000_post_disaster.png"}}